In [ ]:
from Lab_Analyses.Utilities.get_dFoF import get_dFoF
from Lab_Analyses.Utilities.load_mat_files import load_mat

In [ ]:
SnFr_path = r"Z:\People\Jake\Imaging_Data\JW033\220103\dendrite_1\summed"
SnFr_file = "JW033_0103_00001_00001_summed_50_Analyzed_ByJake"

SnFr_data = load_mat(SnFr_file, SnFr_file, SnFr_path)

In [ ]:
SnFr_fluoro = SnFr_data.Fluorescence_Measurement

In [ ]:
SnFr_sampling_rate = 58.30


In [ ]:
SnFr_fluoro[0]

In [ ]:
SnFr_dFoF, SnFr_PdFoF, SnFr_baseline = get_dFoF(SnFr_fluoro[0],SnFr_sampling_rate, smooth_window=0.5)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
plt.figure()
plt.plot(SnFr_dFoF)
plt.figure()
plt.plot(SnFr_PdFoF)
plt.figure()
plt.plot(SnFr_fluoro[0])
plt.plot(SnFr_baseline)

In [ ]:
import numpy as np

new_fluo = (SnFr_fluoro[0] - SnFr_fluoro[0].min()) / (SnFr_fluoro[0].max() - SnFr_fluoro[0].min())
new_base = (SnFr_baseline -  SnFr_fluoro[0].min()) / (SnFr_fluoro[0].max() - SnFr_fluoro[0].min())

In [ ]:
plt.figure()
plt.plot(new_fluo)
plt.plot(new_base)

In [ ]:
c = np.array([5,6,7,8]).reshape(1,-1)
a = np.append(a,c,axis=0)
a[1:,:]

In [ ]:
a = [np.array([1,2,3]), np.array([4,5,6])]
b = np.vstack(a).T
b

In [ ]:
b.mean(axis=1).reshape(-1,1)

In [ ]:
c = np.array([1,2,3]).reshape(-1,1)
d = np.array([4,5,6]).reshape(-1,1)
e = [c,d]
f = np.hstack(e).mean(axis=1).reshape(-1,1)

In [ ]:
file1 = "JW033_0103_00001_00001_corrected"
file2 = "JW033_0103_00001_00002_corrected"
file3 = "JW033_0103_00001_00003_corrected"
file4 = "JW033_0103_00002_00001_corrected"
file5 = "JW033_0103_00002_00002_corrected"
file6 = "JW033_0103_00002_00003_corrected"
files = [file1, file2, file3, file4, file5, file6]
frame_counts = [800,800,650,800,800,440]

In [ ]:
import re

name = "JW015_1113_00002_00001_summed_50"
a = re.search("[A-Z]{2}[0-9]{3}", name).group()
a

In [2]:
import pickle
with open("C:\Users\Jake\Desktop\test_output.pickle", "rb") as f:
    file = pickle.load(f)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_17324/1361557119.py, line 2)

In [ ]:
a = Object(name="Jake")

In [ ]:
import numpy as np

ops = np.load("ops.npy", allow_pickle=True)

In [ ]:
ops